In [2]:
import numpy as np
import pandas as pd

In [20]:
# import the csv file
df = pd.read_csv('journal_comparison_Oct_backtest.csv', header=0, names=['run_date', 'entry'])
df

,run_date,entry
0,2024.02.23 07:19:42.216,"2023.10.16 00:00:00 min_impulse_psar EURUSD,M..."
1,2024.02.23 07:19:42.216,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
2,2024.02.23 07:19:42.216,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
3,2024.02.23 07:19:42.216,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
4,2024.02.23 07:19:42.216,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
...,...,...
10735,2024.02.23 07:20:01.556,"2023.11.09 23:45:00 min_impulse_psar EURUSD,M..."
10736,2024.02.23 07:20:01.556,"2023.11.09 23:45:00 min_impulse_psar EURUSD,M..."
10737,2024.02.23 07:20:01.556,"2023.11.09 23:45:00 min_impulse_psar EURUSD,M..."
10738,2024.02.23 07:20:01.565,2023.11.09 23:59:47 Tester: order #101 is closed


In [23]:
df_drop = df.drop('run_date', axis=1)
df_drop

,entry
0,"2023.10.16 00:00:00 min_impulse_psar EURUSD,M..."
1,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
2,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
3,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
4,"2023.10.16 00:15:00 min_impulse_psar EURUSD,M..."
...,...
10735,"2023.11.09 23:45:00 min_impulse_psar EURUSD,M..."
10736,"2023.11.09 23:45:00 min_impulse_psar EURUSD,M..."
10737,"2023.11.09 23:45:00 min_impulse_psar EURUSD,M..."
10738,2023.11.09 23:59:47 Tester: order #101 is closed


In [24]:
# pull out the datetime from the string
df_drop['datetime'] = df_drop['entry'].str.extract(r'(\d{4}.\d{2}.\d{2} \d{2}:\d{2}:\d{2})')
# remove the datetime from the string
df_drop['entry'] = df_drop['entry'].str.split('min_impulse_psar EURUSD,M15:').str[1]
df_drop

,entry,datetime
0,New Account Equity at Reset: 100000.0,2023.10.16 00:00:00
1,Entering into OnBar,2023.10.16 00:15:00
2,Time: 0,2023.10.16 00:15:00
3,No Trades Allowed,2023.10.16 00:15:00
4,more_trades: false,2023.10.16 00:15:00
...,...,...
10735,SMI: -44.36428143293188,2023.11.09 23:45:00
10736,NO TRADE,2023.11.09 23:45:00
10737,First Cross: true,2023.11.09 23:45:00
10738,NaN,2023.11.09 23:59:47


In [57]:
# remove any rows with NaN
df_drop = df_drop.dropna()
# filter entry to where the beginning of the string includes "Closing"
df_closing = df_drop[df_drop['entry'].str.contains('Closing')]
# reverse the order of the datetime and entry columns
df_closing = df_closing.loc[:, ['datetime', 'entry']]
# # remove the ":00" from the datetime
# df_closing['datetime'] = df_closing['datetime'].str.replace(':00', '')
# # if there is no colon, add ":00" to the datetime
# df_closing['datetime'] = df_closing['datetime'].apply(lambda x: x if ':' in x else x + ':00')
df_closing

,datetime,entry
337,2023.10.16 12:45:00,Closing Buy Ticket: 3 bar rule Ticket #: 1
373,2023.10.16 14:15:00,Closing Buy Ticket: PSAR flip Ticket #: 2
443,2023.10.16 17:30:00,Closing Sell Ticket: SMI crossing above -40 T...
479,2023.10.16 19:00:00,Closing Buy Ticket: SMI crossing below 40 Tic...
588,2023.10.16 23:45:00,Closing Buy Ticket: 3 bar rule Ticket #: 5
...,...,...
9979,2023.11.08 16:00:00,Closing Sell Ticket: SMI crossing above -40 T...
10167,2023.11.09 00:15:00,Closing Buy Ticket: SMI crossing below 40 Tic...
10458,2023.11.09 10:45:00,Closing Sell Ticket: SMI crossing above -40 T...
10538,2023.11.09 14:30:00,Closing Sell Ticket: SMI crossing above -40 T...


In [58]:
# convert the datetime to a datetime object
df_closing['datetime'] = pd.to_datetime(df_closing['datetime'], format='%Y.%m.%d %H:%M:%S')
# subtract 7 hrs from the datetime
# df_closing['datetime_EST'] = pd.to_datetime(df_closing['datetime'], format='%Y.%m.%d %H:%M') - pd.Timedelta(hours=7)
df_closing

,datetime,entry
337,2023-10-16 12:45:00,Closing Buy Ticket: 3 bar rule Ticket #: 1
373,2023-10-16 14:15:00,Closing Buy Ticket: PSAR flip Ticket #: 2
443,2023-10-16 17:30:00,Closing Sell Ticket: SMI crossing above -40 T...
479,2023-10-16 19:00:00,Closing Buy Ticket: SMI crossing below 40 Tic...
588,2023-10-16 23:45:00,Closing Buy Ticket: 3 bar rule Ticket #: 5
...,...,...
9979,2023-11-08 16:00:00,Closing Sell Ticket: SMI crossing above -40 T...
10167,2023-11-09 00:15:00,Closing Buy Ticket: SMI crossing below 40 Tic...
10458,2023-11-09 10:45:00,Closing Sell Ticket: SMI crossing above -40 T...
10538,2023-11-09 14:30:00,Closing Sell Ticket: SMI crossing above -40 T...


In [61]:
# bring in the other csv file
time_df = pd.read_excel('time.xlsx')
# convert the time column to a datetime object
time_df['Time_EST'] = pd.to_datetime(time_df['Time'], format='%Y.%m.%d %H:%M') - pd.Timedelta(hours=7)
time_df['Time'] = pd.to_datetime(time_df['Time'], format='%Y.%m.%d %H:%M')
cols = time_df.columns.tolist()
# make Time_EST the 2nd column
cols = cols[-1:] + cols[:-1]
time_df = time_df.loc[:, cols]
time_df

,Time_EST,Time,action,Order,lot size,Price,S/L,TP,Profit,Balance
0,2023-10-16 04:45:00,2023-10-16 11:45:00,buy,1,7,1.05364,1.05254,0,0.0,100000.00
1,2023-10-16 05:00:00,2023-10-16 12:00:00,modify,1,7,1.05364,1.05294,0,0.0,100000.00
2,2023-10-16 05:45:00,2023-10-16 12:45:00,close,1,7,1.05377,1.05294,0,91.0,100091.00
3,2023-10-16 06:30:00,2023-10-16 13:30:00,buy,2,7,1.05311,1.05201,0,0.0,100091.00
4,2023-10-16 06:45:00,2023-10-16 13:45:00,modify,2,7,1.05311,1.05230,0,0.0,100091.00
...,...,...,...,...,...,...,...,...,...,...
325,2023-11-09 10:02:00,2023-11-09 17:02:00,s/l,99,7,1.07133,1.07133,0,399.0,102689.57
326,2023-11-09 10:45:00,2023-11-09 17:45:00,buy,100,7,1.07152,1.07042,0,0.0,102689.57
327,2023-11-09 11:15:00,2023-11-09 18:15:00,close,100,7,1.07113,1.07042,0,-273.0,102416.57
328,2023-11-09 16:00:00,2023-11-09 23:00:00,sell,101,7,1.06638,1.06748,0,0.0,102416.57


In [62]:
# join the two dataframes on the datetime
df_joined = time_df.set_index('Time').join(df_closing.set_index('datetime'), how='left')
# fill in the 'entry' column with "stoploss" if 's/l' is in the 'action' column
df_joined['entry'] = np.where(df_joined['action'].str.contains('s/l'), 'stoploss', df_joined['entry'])
# reset the index and rename it to 'datetime'
df_reindex = df_joined.reset_index().rename(columns={'index': 'datetime'})
df_reindex

,datetime,Time_EST,action,Order,lot size,Price,S/L,TP,Profit,Balance,entry
0,2023-10-16 11:45:00,2023-10-16 04:45:00,buy,1,7,1.05364,1.05254,0,0.0,100000.00,NaN
1,2023-10-16 12:00:00,2023-10-16 05:00:00,modify,1,7,1.05364,1.05294,0,0.0,100000.00,NaN
2,2023-10-16 12:45:00,2023-10-16 05:45:00,close,1,7,1.05377,1.05294,0,91.0,100091.00,Closing Buy Ticket: 3 bar rule Ticket #: 1
3,2023-10-16 13:30:00,2023-10-16 06:30:00,buy,2,7,1.05311,1.05201,0,0.0,100091.00,NaN
4,2023-10-16 13:45:00,2023-10-16 06:45:00,modify,2,7,1.05311,1.05230,0,0.0,100091.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
325,2023-11-09 17:02:00,2023-11-09 10:02:00,s/l,99,7,1.07133,1.07133,0,399.0,102689.57,stoploss
326,2023-11-09 17:45:00,2023-11-09 10:45:00,buy,100,7,1.07152,1.07042,0,0.0,102689.57,NaN
327,2023-11-09 18:15:00,2023-11-09 11:15:00,close,100,7,1.07113,1.07042,0,-273.0,102416.57,Closing Buy Ticket: SMI crossing below 40 Tic...
328,2023-11-09 23:00:00,2023-11-09 16:00:00,sell,101,7,1.06638,1.06748,0,0.0,102416.57,NaN


In [63]:
# save df_closing to a csv file
df_reindex.to_csv('journal_comparison_Oct_backtest_closing.csv', index=False)